In [1]:
import pandas as pd
# import matplotlib.pyplot as plt
import numpy as np
# import seaborn as sns
# from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.model_selection import cross_val_score
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.tree import DecisionTreeRegressor
# import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
# from scipy import stats

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB

import nltk
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [15]:
import warnings 
warnings.filterwarnings('ignore')

In [4]:
path="https://github.com/deepu-san/Projects/blob/main/Amaon%20Product%20Review/Flipkart_Amazon%20Mobile%20Reviews.csv?raw=true"
df = pd.read_csv(path)
df.head(3)
#df.shape

,Unnamed: 0,Review-Title,rating,Review-Body,Product Name
0,0,Worst phone ever,1.0 out of 5 stars,Hang problem,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12..."
1,1,Ok !!! Not up to the mark,2.0 out of 5 stars,I'm writing this review after using 3days !!!B...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12..."
2,2,Awesome look,5.0 out of 5 stars,Camera is so good n very fast phone back look ...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12..."


In [5]:
df.dtypes

Unnamed: 0       int64
Review-Title    object
rating          object
Review-Body     object
Product Name    object
dtype: object

In [6]:
df.isna().sum()

Unnamed: 0        0
Review-Title      2
rating            0
Review-Body     617
Product Name      0
dtype: int64

In [7]:
df['rating'].value_counts()

5.0 out of 5 stars    9399
1.0 out of 5 stars    5231
4.0 out of 5 stars    4886
3.0 out of 5 stars    2703
2.0 out of 5 stars    1558
Name: rating, dtype: int64

In [8]:
df['Product Name'].value_counts()

Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Storage)           4460
OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)              4330
Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM                 3768
OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 128GB Storage)    3570
Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)            2724
realme narzo 50A (Oxygen Blue , 4GB RAM + 64 GB Storage)      2166
vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128GB Storage)      1419
Samsung Galaxy M32                                            1340
Name: Product Name, dtype: int64

In [9]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [10]:
df=df.dropna()
df= df.reset_index(drop=True)


In [11]:
df['rating_processed'] = (df['rating'].str[:3]).astype('float')
df.head(2)

,Review-Title,rating,Review-Body,Product Name,rating_processed
0,Worst phone ever,1.0 out of 5 stars,Hang problem,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",1.0
1,Ok !!! Not up to the mark,2.0 out of 5 stars,I'm writing this review after using 3days !!!B...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",2.0


In [12]:
df['rating_nature'] =df['rating_processed'].apply(lambda x: 1 if x>2.5 else 0)
df.head(2)

,Review-Title,rating,Review-Body,Product Name,rating_processed,rating_nature
0,Worst phone ever,1.0 out of 5 stars,Hang problem,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",1.0,0
1,Ok !!! Not up to the mark,2.0 out of 5 stars,I'm writing this review after using 3days !!!B...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",2.0,0


# Processing

In [16]:
df['body_processed'] = df['Review-Body'].str.replace("[^a-zA-Z0-9]", " ")
df['body_processed'] = df['body_processed'].apply(lambda row: ' '.join([word for word in row.split() if len(word)>2 or word in ['UI']]))
df['body_processed'] = [review.lower() for review in df['body_processed']]

df['title_processed'] = df['Review-Title'].str.replace("[^a-zA-Z0-9]", " ")
df['title_processed'] = df['title_processed'].apply(lambda row: ' '.join([word for word in row.split() if len(word)>2 or word in ['UI']]))
df['title_processed'] = [review.lower() for review in df['title_processed']]


In [17]:
df[['Review-Title','title_processed', 'Review-Body','body_processed']]

,Review-Title,title_processed,Review-Body,body_processed
0,Worst phone ever,worst phone ever,Hang problem,hang problem
1,Ok !!! Not up to the mark,not the mark,I'm writing this review after using 3days !!!B...,writing this review after using 3days honest t...
2,Awesome look,awesome look,Camera is so good n very fast phone back look ...,camera good very fast phone back look awesomew...
3,One plus losing is originality!!!,one plus losing originality,The media could not be loaded.\n ...,the media could not loaded average product dec...
4,Read,read,I got my delivery on 23 feb when I unboxed the...,got delivery feb when unboxed the phone the di...
...,...,...,...,...
23154,Batten in this rangee,batten this rangee,Must he Bhai iss range me bas cemra thoda sa s...,must bhai iss range bas cemra thoda shi nhi
23155,Good mobile,good mobile,Good product,good product
23156,Good camera,good camera,Good,good
23157,Warest mobile,warest mobile,Don't buy this mobile phone money weast screen...,don buy this mobile phone money weast screen t...


In [18]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwords.words('english')


def remove_stopwords(rev):
    # iNPUT : IT WILL TAKE ROW/REVIEW AS AN INPUT
    # take the paragraph, break into words, check if the word is a stop word, remove if stop word, combine the words into a para again
    review_tokenized = word_tokenize(rev)
    rev_new = " ".join([i for i in review_tokenized  if i not in stop_words])
    return rev_new

# Removing stopwords
df['body_processed'] = [remove_stopwords(r) for r in df['body_processed']]
df['title_processed'] = [remove_stopwords(r) for r in df['title_processed']]



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Deepu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Deepu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [19]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') # Run atleast once 

lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')

# Finds the part of speech tag
# Convert the detailed POS tag into a shallow information
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

# lemmatize sentence using pos tag
def lemmatize_sentence(sentence):
  # word tokenize -> pos tag (detailed) -> wordnet tag (shallow pos) -> lemmatizer -> root word
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)


df['body_processed'] = df['body_processed'].apply(lambda x: lemmatize_sentence(x))
df['title_processed'] = df['title_processed'].apply(lambda x: lemmatize_sentence(x))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Deepu\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Deepu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Deepu\AppData\Roaming\nltk_data...


# Bagging of Words

In [20]:
tfidf = TfidfVectorizer(max_features=None)

x1 = tfidf.fit_transform(df.body_processed).toarray()
x2 = tfidf.fit_transform(df.title_processed).toarray()

X=np.concatenate((x1,x2), axis=1)

y = df.rating_nature.values

In [21]:
print(x1.shape)
print(x2.shape)
print(X.shape)

(23159, 12781)
(23159, 4316)
(23159, 17097)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)


# Decision Tree

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

y_pred = dt.predict(X_test)

In [ ]:
y_pred.shape

(4632,)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(cm)
print("The model accuracy is", accuracy )

[[ 901  430]
 [ 385 2916]]
The model accuracy is 0.8240500863557858


In [ ]:
y_test.shape

(4632,)

In [ ]:
#roc_auc_score(y_test, dt.predict_proba(X_test))#,multi_class='ovr'

roc_auc_score(y_test, dt.predict_proba(X_test)[:, 1])

0.7782067952452084

# Multinomial NB

In [32]:
nb = MultinomialNB()
nb.fit(X_train,y_train)

MultinomialNB()

In [33]:
predictions = nb.predict(X_test)

In [34]:
print(confusion_matrix(y_test,predictions))


[[ 935  410]
 [ 220 3067]]


In [35]:
roc_auc_score(y_test, nb.predict_proba(X_test)[:, 1],multi_class='ovr')

0.9256639029725074

In [36]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.81      0.70      0.75      1345
           1       0.88      0.93      0.91      3287

    accuracy                           0.86      4632
   macro avg       0.85      0.81      0.83      4632
weighted avg       0.86      0.86      0.86      4632



# Bernoulli NB

In [28]:
#crashing due to inadequate RAM

from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB()
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)

In [29]:
print(confusion_matrix(y_test,predictions))

[[ 854  491]
 [ 246 3041]]


In [30]:
roc_auc_score(y_test, nb.predict_proba(X_test)[:, 1],multi_class='ovr')

0.9256639029725074

In [31]:
df.head(3)

,Review-Title,rating,Review-Body,Product Name,rating_processed,rating_nature,body_processed,title_processed
0,Worst phone ever,1.0 out of 5 stars,Hang problem,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",1.0,0,hang problem,worst phone ever
1,Ok !!! Not up to the mark,2.0 out of 5 stars,I'm writing this review after using 3days !!!B...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",2.0,0,write review use 3days honest normal android p...,mark
2,Awesome look,5.0 out of 5 stars,Camera is so good n very fast phone back look ...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",5.0,1,camera good fast phone back look awesomewith d...,awesome look


In [37]:
df['rating_nature_pred']=nb.predict(X)

In [38]:
df.head(3)

,Review-Title,rating,Review-Body,Product Name,rating_processed,rating_nature,body_processed,title_processed,rating_nature_pred
0,Worst phone ever,1.0 out of 5 stars,Hang problem,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",1.0,0,hang problem,worst phone ever,0
1,Ok !!! Not up to the mark,2.0 out of 5 stars,I'm writing this review after using 3days !!!B...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",2.0,0,write review use 3days honest normal android p...,mark,0
2,Awesome look,5.0 out of 5 stars,Camera is so good n very fast phone back look ...,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",5.0,1,camera good fast phone back look awesomewith d...,awesome look,1


## Problem 1

In [104]:
df1=pd.DataFrame(df.groupby("Product Name")["rating_nature_pred"].sum())#.reset_index())
df1

,rating_nature_pred
Product Name,
"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)",3426
"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 128GB Storage)",2741
Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM,2491
"Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Storage)",3226
"Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)",1875
Samsung Galaxy M32,868
"realme narzo 50A (Oxygen Blue , 4GB RAM + 64 GB Storage)",1687
"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128GB Storage)",1019


In [105]:
df1.rename(columns = {'rating_nature_pred':'pred_pos'}, inplace = True)
df1

,pred_pos
Product Name,
"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)",3426
"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 128GB Storage)",2741
Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM,2491
"Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Storage)",3226
"Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)",1875
Samsung Galaxy M32,868
"realme narzo 50A (Oxygen Blue , 4GB RAM + 64 GB Storage)",1687
"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128GB Storage)",1019


In [106]:
df1["act_pos"]=df.groupby("Product Name")["rating_nature"].sum().tolist()
df1

,pred_pos,act_pos
Product Name,,
"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)",3426,3254
"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 128GB Storage)",2741,2616
Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM,2491,2311
"Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Storage)",3226,3126
"Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)",1875,1795
Samsung Galaxy M32,868,847
"realme narzo 50A (Oxygen Blue , 4GB RAM + 64 GB Storage)",1687,1595
"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128GB Storage)",1019,924


In [99]:
df.groupby("Product Name")["rating_nature"].sum()#.tolist()

Product Name
OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)              3254
OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 128GB Storage)    2616
Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM                 2311
Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Storage)           3126
Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)            1795
Samsung Galaxy M32                                             847
realme narzo 50A (Oxygen Blue , 4GB RAM + 64 GB Storage)      1595
vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128GB Storage)       924
Name: rating_nature, dtype: int64

In [121]:
df1["tot_rev"]=df.groupby('Product Name')['rating_nature'].count()

df1

,pred_pos,act_pos,tot_rev
Product Name,,,
"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)",3426,3254,4273
"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 128GB Storage)",2741,2616,3396
Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM,2491,2311,3597
"Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Storage)",3226,3126,4450
"Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)",1875,1795,2642
Samsung Galaxy M32,868,847,1336
"realme narzo 50A (Oxygen Blue , 4GB RAM + 64 GB Storage)",1687,1595,2100
"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128GB Storage)",1019,924,1365


In [108]:
df["Product Name"].value_counts()

Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Storage)           4450
OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)              4273
Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM                 3597
OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 128GB Storage)    3396
Redmi Note 11 (Space Black, 4GB RAM, 64GB Storage)            2642
realme narzo 50A (Oxygen Blue , 4GB RAM + 64 GB Storage)      2100
vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128GB Storage)      1365
Samsung Galaxy M32                                            1336
Name: Product Name, dtype: int64

Index(['Product Name', 'pred_pos', 'act_pos', 'tot_rev'], dtype='object')

In [132]:
df1.reset_index(inplace=True)
df1=df1[["Product Name","tot_rev","act_pos","pred_pos"]]
df1

,Product Name,tot_rev,act_pos,pred_pos
0,"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)",4273,3254,3426
1,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",3396,2616,2741
2,Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM,3597,2311,2491
3,"Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Sto...",4450,3126,3226
4,"Redmi Note 11 (Space Black, 4GB RAM, 64GB Stor...",2642,1795,1875
5,Samsung Galaxy M32,1336,847,868
6,"realme narzo 50A (Oxygen Blue , 4GB RAM + 64 G...",2100,1595,1687
7,"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128G...",1365,924,1019


In [134]:
df1["pred_pos_prcnt"]=(df1["pred_pos"]/df1["tot_rev"]*100).round(2)
df1["act_pos_prcnt"]=(df1["act_pos"]/df1["tot_rev"]*100).round(2)
df1

,Product Name,tot_rev,act_pos,pred_pos,pred_pos_prcnt,act_pos_prcnt
0,"OPPO A31 (Mystery Black, 6GB RAM, 128GB Storage)",4273,3254,3426,80.18,76.15
1,"OnePlus Nord CE 2 5G (Gray Mirror, 8GB RAM, 12...",3396,2616,2741,80.71,77.03
2,Redmi 10 Prime (Bifrost Blue 4GB RAM 64GB ROM,3597,2311,2491,69.25,64.25
3,"Redmi 9 Activ (Carbon Black, 4GB RAM, 64GB Sto...",4450,3126,3226,72.49,70.25
4,"Redmi Note 11 (Space Black, 4GB RAM, 64GB Stor...",2642,1795,1875,70.97,67.94
5,Samsung Galaxy M32,1336,847,868,64.97,63.40
6,"realme narzo 50A (Oxygen Blue , 4GB RAM + 64 G...",2100,1595,1687,80.33,75.95
7,"vivo iQOO Z6 5G (Chromatic Blue, 6GB RAM, 128G...",1365,924,1019,74.65,67.69


## Problem 2

In [136]:
cam=["camera"]
bat=["battery"]
disp=["display","screen"]
value=["price","money"]
perf=["performance"]

In [143]:
df['cat'] = 0


In [147]:
df.loc[df['Review-Body'].str.contains('camera'), 'cat'] = 1

In [153]:
pd.set_option("display.max_colwidth", -1)

In [171]:
df.columns

Index(['Review-Title', 'rating', 'Review-Body', 'Product Name',
       'rating_processed', 'rating_nature', 'body_processed',
       'title_processed', 'rating_nature_pred', 'cat'],
      dtype='object')

In [174]:
pd.DataFrame(df.loc[df['cat']==1]['Review-Body'])#,'rating_processed'])#['rating_processes']['rating_nature_pred'])


,Review-Body
3,The media could not be loaded.\n \n\n\n\n It’s A average product decent for day to day activity.. one the thing I would like to mention that exchange was Far Far alot Good On amazon it was like 2 to 3 minutes and delivery agent was very Good with me although I had 2 scratches on it but he neglected it as it was minor scratches and I exchanged poco m2 pro 6 64gb variant which was exchanged for rupees 10100. I bought it at 12499 1 and Half year ago and sold it at at very good price!! So plus point To amazonComing Back to device I would say it’s a ok device camera is average performance is Fine one plus is losing it’s originality oxygen os to color or..WARP CHARGER TO VOOC CHARGER!! yes it has been taken by oppo!! I wish the way we believed in one plus doesn’t change with it as A one plus Fan We don’t want To merge with Other brands please keep your Originality Maintained!! thanx amazon for such a hasselfree exchange alot better than FLIPKART
5,"The media could not be loaded.\n \n\n\n\n Phone is is Fab!But sometime apps like Gaana, LinkedIn, etc are crashing, I think need to fix some bugs from OnePlus end.However, camera is good, finishing quality is good, sound effect could be better(not that load as compared to iPhone or other Mi phones)Touch is awesome, finger print sensor works too fast, excellent face detection unlockMost sexiest thing is charging, it charges way too quickly, 35-40min approx for full chageMany more extra features from Android Oxygen & One plus!!Phone is handy & a good buy in this range!"
6,"Nord CE 2 is a decent choice for someone looking to upgrade from an under 15k segment phone. Review after using the 6GB variant for 5 days:Pros:- Decent battery life and excellent charging speed- Smooth performance and display- Comparatively compact and light weight- ""Unofficial"" water resistance (read below)- Design and color is neatCons:- Speaker isn't great- Camera is okayish- Unsure of OS experience in the future, stable and bloat free as of nowDetailed:- Decent battery life and performance. I got around 7 to 8 hrs of screen time from battery at 100% to 20% with normal use with 4G internet (social media, chat, some camera and mobile hotspot tethering at 90HZ display refresh rate). Changing display refresh rate to 60Hz seems to improve battery drain, but 90Hz looks very smooth.- 65W fast charging is great, can charge from 20% to 100% in 30 minutes. Also not much heating during charging.- OS looks clean, some preinstalled apps could be uninstalled or disabled. But surprisingly found 3 file manager apps (of which two can be disabled) and two photos app (OnePlus Gallery and Google Photos app, which can't be uninstalled). Didn't find any bugs or functionality issue, no lags or stuttering. Apps open instantly and work fine. OxygenOS 11.3 with Feb 2022 security patch. I guess a stable Android 11 is better than a buggier Android 12 for most customers. But not sure what will happen from OxygenOS 12 which shares Oppo's ColorOS.- Some heating issue was there on a first day of use, but then got a system update and haven't noticed any heating afterwards.- Saw a rubber seal for the SIM tray, which could mean that there is some degree of water resistance, but there is no official IP rating. Also saw a YouTube video that tests the water resistance and the phone survived 15 min submersion in water. So there seems to be ""unofficial"" water resistance.- Cameras are okayish. Main cam takes good pictures at good lighting, but colors are a bit saturated. In wide angle camera, the exposure and color is less. Photos from selfie cam are a bit soft without any filters or modes. Haven't tested much low light or night photography.- Phone back panel is made of plastic, feels like plastic to touch but the blue color back panel looks really good. Good to hold in hand and use because of comparatively compact and thin size. A smaller bottom chin / bezel would have been better. The case provided is translucent and has a small pattern

In [181]:
df[['Review-Body','rating_processed','rating_nature_pred']] [df['cat'] == 1]

,Review-Body,rating_processed,rating_nature_pred
3,The media could not be loaded.\n \n\n\n\n It’s A average product decent for day to day activity.. one the thing I would like to mention that exchange was Far Far alot Good On amazon it was like 2 to 3 minutes and delivery agent was very Good with me although I had 2 scratches on it but he neglected it as it was minor scratches and I exchanged poco m2 pro 6 64gb variant which was exchanged for rupees 10100. I bought it at 12499 1 and Half year ago and sold it at at very good price!! So plus point To amazonComing Back to device I would say it’s a ok device camera is average performance is Fine one plus is losing it’s originality oxygen os to color or..WARP CHARGER TO VOOC CHARGER!! yes it has been taken by oppo!! I wish the way we believed in one plus doesn’t change with it as A one plus Fan We don’t want To merge with Other brands please keep your Originality Maintained!! thanx amazon for such a hasselfree exchange alot better than FLIPKART,3.0,1
5,"The media could not be loaded.\n \n\n\n\n Phone is is Fab!But sometime apps like Gaana, LinkedIn, etc are crashing, I think need to fix some bugs from OnePlus end.However, camera is good, finishing quality is good, sound effect could be better(not that load as compared to iPhone or other Mi phones)Touch is awesome, finger print sensor works too fast, excellent face detection unlockMost sexiest thing is charging, it charges way too quickly, 35-40min approx for full chageMany more extra features from Android Oxygen & One plus!!Phone is handy & a good buy in this range!",4.0,1
6,"Nord CE 2 is a decent choice for someone looking to upgrade from an under 15k segment phone. Review after using the 6GB variant for 5 days:Pros:- Decent battery life and excellent charging speed- Smooth performance and display- Comparatively compact and light weight- ""Unofficial"" water resistance (read below)- Design and color is neatCons:- Speaker isn't great- Camera is okayish- Unsure of OS experience in the future, stable and bloat free as of nowDetailed:- Decent battery life and performance. I got around 7 to 8 hrs of screen time from battery at 100% to 20% with normal use with 4G internet (social media, chat, some camera and mobile hotspot tethering at 90HZ display refresh rate). Changing display refresh rate to 60Hz seems to improve battery drain, but 90Hz looks very smooth.- 65W fast charging is great, can charge from 20% to 100% in 30 minutes. Also not much heating during charging.- OS looks clean, some preinstalled apps could be uninstalled or disabled. But surprisingly found 3 file manager apps (of which two can be disabled) and two photos app (OnePlus Gallery and Google Photos app, which can't be uninstalled). Didn't find any bugs or functionality issue, no lags or stuttering. Apps open instantly and work fine. OxygenOS 11.3 with Feb 2022 security patch. I guess a stable Android 11 is better than a buggier Android 12 for most customers. But not sure what will happen from OxygenOS 12 which shares Oppo's ColorOS.- Some heating issue was there on a first day of use, but then got a system update and haven't noticed any heating afterwards.- Saw a rubber seal for the SIM tray, which could mean that there is some degree of water resistance, but there is no official IP rating. Also saw a YouTube video that tests the water resistance and the phone survived 15 min submersion in water. So there seems to be ""unofficial"" water resistance.- Cameras are okayish. Main cam takes good pictures at good lighting, but colors are a bit saturated. In wide angle camera, the exposure and color is less. Photos from selfie cam are a bit soft without any filters or modes. Haven't tested much low light or night photography.- Phone back panel is made of plastic, feels like plastic to touch but the blue color back panel looks really good. Good to hold in hand and use because of comparatively compact and thin size. A smaller bottom chin / bezel would have been better. The case

# Sentiment Analysis

In [ ]:
pd.set_option("display.max_colwidth", -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
nltk.download('vader_lexicon')

def sentiment_analyse(sentiment_text):
    score = SentimentIntensityAnalyzer().polarity_scores(sentiment_text)
    if score['neg'] > score['pos']:
        return ("Negative")
    elif score['neg'] < score['pos']:
        return ("Positive")
    else:
        return ("Neutral")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
sid = SentimentIntensityAnalyzer()
def comp_score(txt):
    score=sid.polarity_scores(txt)
    if score['compound']>0 : return "Pos"
    else : return "Neg"

In [ ]:
df['review_score']=[sentiment_analyse(r) for r in df['Review-Body']]
#df['review_comp']=[comp_score(r) for r in df['Review-Body']]

In [ ]:
df['review_comp']=[comp_score(r) for r in df['Review-Body']]

In [ ]:
df[['Review-Body','review_score','review_comp']]

,Review-Body,review_score,review_comp
0,Hang problem,Negative,Neg
1,I'm writing this review after using 3days !!!Be to honest this is normal Android phoneIt's not like OnePlusCamera quality very low it says 64mp but notSound also lowBattery 🔋 backup ok !!! For one day normal use itsIf ur Gamer don't go for itOverall price it High for this handsetIf u really wanna buy OnePlus ho for higher end modelI'm first time buying OnePlusLittle disappointment,Negative,Neg
2,Camera is so good n very fast phone back look is awesomeWith dis price segment it's worth it n d most important thing is the phone has dedicated memory card slot I think dis is the first one plus phone which has d dedicated slot for memory cardBattery backup is also good n wit the help of 65w charger u can charge ur phone frm 50-100% in just 15 minsIt has the super vooc charger frm Dat u can charge ur phone very fastVery smooth touch n very fast phone u can play games with no lagInternet connectivity is also fast n lightning wit my Jio sim as u can c I hav pasted one photo showing d downloading speed of 11mbps wit my cellular networkI think everyone who wants to buy dis phone just go for itI think oneplus launched dis phone with lots of good features n d most important thing is price is very lessI recommendJust go for itThank u oneplus n Amazon for dis wonderful product,Positive,Pos
3,The media could not be loaded.\n \n\n\n\n It’s A average product decent for day to day activity.. one the thing I would like to mention that exchange was Far Far alot Good On amazon it was like 2 to 3 minutes and delivery agent was very Good with me although I had 2 scratches on it but he neglected it as it was minor scratches and I exchanged poco m2 pro 6 64gb variant which was exchanged for rupees 10100. I bought it at 12499 1 and Half year ago and sold it at at very good price!! So plus point To amazonComing Back to device I would say it’s a ok device camera is average performance is Fine one plus is losing it’s originality oxygen os to color or..WARP CHARGER TO VOOC CHARGER!! yes it has been taken by oppo!! I wish the way we believed in one plus doesn’t change with it as A one plus Fan We don’t want To merge with Other brands please keep your Originality Maintained!! thanx amazon for such a hasselfree exchange alot better than FLIPKART,Positive,Pos
4,I got my delivery on 23 feb when I unboxed the phone the display was damaged. I requested for replacement of my new phone so technician was suppose to come for inspection but instead of a visit he just talked on call and asked me to install an application. He did not even saw the damage of my display screen.Lets hope amazon does it right replace my phone. Will update the rest later.But amazon pls consider my request and replace the item as soon as possibleEdit- The phone is perfect... They replaced it and now everything is perfect...Camera quality is as expected very perfectOverall performance bestYou can go for it,Positive,Pos
...,...,...,...
23154,Must he Bhai iss range me bas cemra thoda sa shi nhi he,Neutral,Neg
23155,Good product,Positive,Pos
23156,Good,Positive,Pos
23157,"Don't buy this mobile phone money weast screen tach problem,no camera quality, no battery 🔋 life, Phone very slow, please don't buy this mobile phone please 🙏🙏🙏🙏🙏",Positive,Pos


In [ ]:
sentiment_analyse("Don't buy this mobile phone money weast screen tach problem,no camera quality, no battery life,Phone very slow,")# please don't buy this mobile phone please ")

'Negative'

In [ ]:
a = 'This was a bad movie.'
sid.polarity_scores(a)

{'compound': -0.5423, 'neg': 0.538, 'neu': 0.462, 'pos': 0.0}

In [ ]:
sid.polarity_scores('The shoes I brought were amazing.')


{'compound': 0.5859, 'neg': 0.0, 'neu': 0.513, 'pos': 0.487}

In [ ]:
sentiment_analyse("please don't buy this mobile phone please")

'Positive'

In [ ]:
sentiment_analyse("The shoes I brought were amazing.")

'Positive'

In [ ]:
comp_score('The shoes I brought were amazing.')

'Pos'

In [ ]:
comp_score("please don't buy this mobile phone please")

'Pos'

In [ ]:
txt1='Must he Bhai iss range me bas cemra thoda sa shi nhi he'

[]